<a href="https://colab.research.google.com/github/Nadil-K/DataStorm4.0/blob/main/DataStorm_Semi_Prepped_for_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
import numpy as np
import pandas as pd

freezer_data=pd.read_csv('freezer_data.csv')  #df1
outlet_data=pd.read_csv('outlets_data.csv')   #df2
product_data=pd.read_csv('product_data.csv')  #df3
week_data=pd.read_csv('week_data.csv')        #df4
sales_data=pd.read_csv('sales_data.csv')      #df5

In [92]:
freezer_data = freezer_data.drop(columns=['Freezer Name'], axis=1)  #df11
product_data = product_data.drop(columns=['product_name'], axis=1)  #df31
product_data['product_type'] = product_data['pid'].str[0]
product_data['product_type'] = product_data['product_type'].map({'I':0, 'B':1})

In [93]:
outlet_data = outlet_data.sort_values('area(sqft)', ascending=False)
outlet_data.drop_duplicates(subset=['Outlet_ID'], keep='first', inplace=True)

In [94]:
sales_data=sales_data.drop("product_name", axis=1)
sales_data["no_units"]=sales_data["no_units"].astype(int)
sales_data = sales_data.drop_duplicates()

In [95]:
merged_outlet_sales = pd.merge(outlet_data, sales_data[['Outlet_ID', 'week', 'pid', 'no_units']], on='Outlet_ID')

general_revenue = pd.merge(merged_outlet_sales, product_data[['pid', 'price']], on='pid')
general_revenue['revenue'] = general_revenue['no_units']*general_revenue['price']

total_revenue_per_store = general_revenue.groupby('Outlet_ID')[['revenue','no_units']].sum().reset_index()

In [98]:
# Merge sales data with product data
# ignore volume and product name
temp_df = pd.merge(product_data[['pid','price']], sales_data, on = ['pid'], how = 'right')

# merge outlet data with new data 
x_df = pd.merge(temp_df, outlet_data, on = ['Outlet_ID'], how = 'left')

#product type
x_df['product_type'] = x_df['pid'].str[0]
x_df['product_type'] = x_df['product_type'].map({'I':0, 'B':1})

#bins
bins = [0, 4, 7, 10, 13]
labels = ['jan', 'feb', 'mar', 'apr']

x_df['month'] = pd.cut(x_df['week'], bins, labels=labels)
x_df = x_df.drop('week',axis=1)

x_df

#creating Columns
temp = pd.DataFrame()
temp = x_df[['Outlet_ID', 'no_units', 'month', 'product_type']]

grouped_data = temp.groupby(['Outlet_ID', 'month', 'product_type'])['no_units'].sum().reset_index()
grouped_data['product_type'] = grouped_data['product_type'].astype(str)
grouped_data['sale_per_month_by_type'] = grouped_data['month'].str.cat(grouped_data['product_type'], sep='_type_')
grouped_data

#Polishup
pivot_df = grouped_data.pivot_table(index='Outlet_ID', columns='sale_per_month_by_type', values='no_units')

# Reset column index and convert to DataFrame
pivot_df = pivot_df.reset_index().rename_axis(None, axis=1)

In [99]:
merged_df = pd.merge(outlet_data, total_revenue_per_store[['Outlet_ID', 'revenue']], on='Outlet_ID')
merged_df = pd.merge(merged_df, pivot_df, on='Outlet_ID')

merged_df

,Outlet_ID,area(sqft),revenue,apr_type_0,apr_type_1,feb_type_0,feb_type_1,jan_type_0,jan_type_1,mar_type_0,mar_type_1
0,ID1772,9950,14446520,6457,1896,11481,2730,15093,2527,11225,1938
1,ID3551,9912,13455030,5350,1468,9377,1932,12622,2733,12163,2440
2,ID7758,9910,13393550,7224,1459,9327,1332,16129,3043,12854,2201
3,ID5341,9896,6848500,0,1220,0,1504,0,2130,0,1362
4,ID3289,9894,13428950,7795,1219,10299,1971,15815,2895,8423,2252
...,...,...,...,...,...,...,...,...,...,...,...
946,ID7472,264,2298000,4220,0,5670,0,7028,0,6012,0
947,ID1494,152,2035590,2538,0,5955,0,8013,0,3755,0
948,ID9777,151,1886390,2871,0,4816,0,6706,0,4524,0
949,ID2283,125,1672170,2694,0,3256,0,5971,0,4860,0
